In [1]:
!pip3 install tensorflow_addons

In [2]:
import sys
sys.path.append('/home/jovyan/ChestXray-14')

In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_addons as tfa

from modules.utils import get_dataset
from modules.dataset import LABELS

2023-04-10 03:37:54.136659: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
/opt/conda/lib/python3.10/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
/opt/conda/lib/python3.10/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal 

In [4]:
import os
import pprint

In [5]:
from pathlib import Path

In [6]:
CURRENT_PATH = os.path.abspath("")
CURRENT_PATH

'/home/jovyan/ChestXray-14/experiments/MultiClass-Classification'

In [7]:
num_class = 15 # TODO: change to 15 for multi-labels
ROOT_PATH = "/home/jovyan/ChestXray-14"
INPUT_PATH = f"{ROOT_PATH}/dataset/ChestXray NIH"
EXPERIMENT_NAME = "multiclass_classification_with_cross_entropy_loss"

In [8]:
class Dataset:
    INPUT_PATH = INPUT_PATH
    
    def __init__(self, fold_num):
        self.fold_num = fold_num
    
    def get_train(self):
        filenames = tf.io.gfile.glob(f'{self.INPUT_PATH}/data/multiclass_dataset/folds/fold{self.fold_num}/train/*.tfrec')
        dataset = get_dataset(filenames)
        return dataset

    def get_test(self):
        filenames = tf.io.gfile.glob(f'{self.INPUT_PATH}/data/multiclass_dataset/folds/fold{self.fold_num}/test/*.tfrec')
        dataset = get_dataset(filenames)
        return dataset

## Experiment

In [9]:
class Evaluate:
    def __init__(self, model_path):
        self.y_true = None
        self.y_preds = None
        self.model_path = model_path
        self.model = self.get_model(model_path)

    def get_model(self, path):
        return tf.keras.models.load_model(path)

    def get_y_true(self, data):
        y_true=[]
        for X,y in data:
            for label in y:
                y_true.append(label)
        y_true = tf.Variable(y_true)
        self.y_true = y_true
        return y_true
    def model_predict(self, test_dataset):
        return self.model.predict(test_dataset)
    

## Using

In [10]:
from tqdm.notebook import tqdm

In [11]:
import time

class TimeUsed:
    def start(self):
        self.start = time.time()
     
    def stop_and_report(self):
        self.end = time.time()
        print(
            "===== " * 5,
            "Model used for: {:.2f} second(s)".format(self.end - self.start),
            "===== " * 5,
            "\n\n"
        )

In [12]:
EXPERIMENT_NAME

'multiclass_classification_with_cross_entropy_loss'

In [13]:
# test_dataset = Dataset(fold_num).get_test()
# MODEL_PATH = f'{CURRENT_PATH}/results/models/{EXPERIMENT_NAME}/EfficientNetB0_None_FOLD_{fold_num}.h5'

# model = tf.keras.models.load_model(MODEL_PATH)
# model = tf.config.run_functions_eagerly(True)
# model.predict(test_dataset)

In [14]:
for fold_num in tqdm(range(1, 5+1)):
    time_counter = TimeUsed()
    time_counter.start()
    
    # Record time for training
    MODEL_PATH = f'{CURRENT_PATH}/results/models/{EXPERIMENT_NAME}/EfficientNetB0_None_FOLD_{fold_num}.h5'
    RESULT_EVALUATE_PATH = os.path.join(CURRENT_PATH, "results", "evaluate", EXPERIMENT_NAME, "EfficientNetB0_None", "Folds", f"fold_{fold_num}")
    Path(RESULT_EVALUATE_PATH).mkdir(parents=True, exist_ok=True)
    
    test_dataset = Dataset(fold_num).get_test()
    
    best_model = Evaluate(MODEL_PATH)
    
    y_true = best_model.get_y_true(test_dataset)
    y_preds = best_model.model_predict(test_dataset)
    target_names = ["Infiltration", "Atelectasis","Effusion", "No Finding"]
    
    print(classification_report(y_true, y_pred, target_names=target_names))
    
#     with best_model:
#         f1_each_class = best_model.get_f1_scores(test_dataset)
#         pprint.pprint(f1_each_class)
#         df = pd.DataFrame(f1_each_class)
#         df.to_csv(f"{RESULT_EVALUATE_PATH}/f1_scores.csv", index=False)

#         precision_each_class = best_model.get_precision_scores(test_dataset, new_calculate=False)
#         pd.DataFrame(precision_each_class)\
#             .to_csv(f"{RESULT_EVALUATE_PATH}/precision.csv", index=False)

#         recall_each_class = best_model.get_recall_scores(test_dataset, new_calculate=False)
#         pd.DataFrame(recall_each_class)\
#             .to_csv(f"{RESULT_EVALUATE_PATH}/recall.csv", index=False)

        # print(df)
    
    time_counter.stop_and_report()

  0%|          | 0/5 [00:00<?, ?it/s]

2023-04-10 03:37:56.267981: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-10 03:37:57.031886: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 8011 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB MIG 2g.10gb, pci bus id: 0000:31:00.0, compute capability: 8.0


INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA A100-SXM4-40GB MIG 2g.10gb, compute capability 8.0


ValueError: Unexpected result of `predict_function` (Empty batch_outputs). Please use `Model.compile(..., run_eagerly=True)`, or `tf.config.run_functions_eagerly(True)` for more information of where went wrong, or file a issue/bug to `tf.keras`.

In [15]:
test_dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 4), dtype=tf.int64, name=None))>

In [16]:
tf.config.run_functions_eagerly(True)
best_model.model.predict(test_dataset)

ValueError: Unexpected result of `predict_function` (Empty batch_outputs). Please use `Model.compile(..., run_eagerly=True)`, or `tf.config.run_functions_eagerly(True)` for more information of where went wrong, or file a issue/bug to `tf.keras`.